In [ ]:
# cd .\WorkingDirectory\LocalstackDockerCompose
# docker-compose up -d --build

In [16]:
docker ps --filter="name=localstack_main"

CONTAINER ID   IMAGE                          COMMAND                  CREATED              STATUS                    PORTS                                                                                                                                                           NAMES
af025721bfb5   localstack/localstack:latest   "docker-entrypoint.sh"   About a minute ago   Up 58 seconds (healthy)   0.0.0.0:53->53/tcp, 0.0.0.0:443->443/tcp, 0.0.0.0:4566->4566/tcp, 4510-4559/tcp, 0.0.0.0:4571->4571/tcp, 0.0.0.0:8080->8080/tcp, 0.0.0.0:53->53/udp, 5678/tcp   localstack_main


[aws localstack and Terraform](https://docs.localstack.cloud/user-guide/integrations/terraform/)

In [ ]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$WorkingDirectory = $OriginalProjectPath + '\WorkingDirectory\LocalstackAwsExample'
"OriginalProjectPath: $OriginalProjectPath"
"WorkingDirectory: $WorkingDirectory"

In [ ]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName
"TerraformExePath: $TerraformExePath"

In [17]:
.\terraform version

Terraform v1.4.2
on windows_amd64


In [31]:
$command = ".\terraform -chdir=$WorkingDirectory init -input=false"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalstackAwsExample init -input=false

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v3.46.0

╷
│ Warning: Version constraints inside provider configuration blocks are deprecated
│ 
│   on LocalstackAwsExample.tf line 2, in provider "aws":
│    2:   version = "= 3.46.0"
│ 
│ Terraform 0.13 and earlier allowed provider version constraints inside the
│ provider configuration block, but that is now deprecated and will be
│ removed in a future version of Terraform. To silence this warning, move the
│ provider version constraint into the required_providers block.
╵

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Te

In [32]:
$command = ".\terraform -chdir=$WorkingDirectory plan -input=false"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalstackAwsExample plan -input=false
aws_kinesis_stream.eaten_hot_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/eatenHotDogs]
aws_kinesis_stream.caught_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/caughtDogs]
aws_kinesis_stream.hot_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/hotDogs]
aws_dynamodb_table.dogs: Refreshing state... [id=dogs]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_dynamodb_table.dogs will be created
  + resource "aws_dynamodb_table" "dogs" {
      + arn              = (known after apply)
      + billing_mode     = "PROVISIONED"
      + hash_key         = "ID"
      + id

In [33]:
$command = ".\terraform -chdir=$WorkingDirectory apply --auto-approve"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalstackAwsExample apply --auto-approve
aws_kinesis_stream.hot_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/hotDogs]
aws_kinesis_stream.caught_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/caughtDogs]
aws_kinesis_stream.eaten_hot_dogs_stream: Refreshing state... [id=arn:aws:kinesis:ap-southeast-2:000000000000:stream/eatenHotDogs]
aws_dynamodb_table.dogs: Refreshing state... [id=dogs]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_dynamodb_table.dogs will be created
  + resource "aws_dynamodb_table" "dogs" {
      + arn              = (known after apply)
      + billing_mode     = "PROVISIONED"
      + hash_key         = "ID"
      +

In [34]:
$command = "awslocal dynamodb list-tables"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
awslocal dynamodb list-tables
{
    "TableNames": []
}


In [ ]:
if (!$IsWindows) {exit} # Guard Clause this section is only for Windows

In [47]:
$command = @"
awslocal dynamodb create-table 
    --table-name Music 
    --attribute-definitions 
        AttributeName=Artist,AttributeType=S 
        AttributeName=SongTitle,AttributeType=S 
    --key-schema 
        AttributeName=Artist,KeyType=HASH 
        AttributeName=SongTitle,KeyType=RANGE 
    --provisioned-throughput 
        ReadCapacityUnits=5,WriteCapacityUnits=5 
    --table-class STANDARD
"@
Write-Output "Executing the command:"
$command = $command.replace("`n","").replace("`r","")
Write-Output $command
Invoke-Expression $command

Executing the command:
awslocal dynamodb create-table     --table-name Music     --attribute-definitions         AttributeName=Artist,AttributeType=S         AttributeName=SongTitle,AttributeType=S     --key-schema         AttributeName=Artist,KeyType=HASH         AttributeName=SongTitle,KeyType=RANGE     --provisioned-throughput         ReadCapacityUnits=5,WriteCapacityUnits=5     --table-class STANDARD
{
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "Artist",
                "AttributeType": "S"
            },
            {
                "AttributeName": "SongTitle",
                "AttributeType": "S"
            }
        ],
        "TableName": "Music",
        "KeySchema": [
            {
                "AttributeName": "Artist",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "SongTitle",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus

In [48]:
$command = "awslocal dynamodb scan --endpoint-url http://localhost:4566 --table-name Music"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
awslocal dynamodb scan --endpoint-url http://localhost:4566 --table-name Music
{
    "Items": [],
    "Count": 0,
    "ScannedCount": 0,
    "ConsumedCapacity": null
}


In [ ]:
# docker stop localstack_main